In [ ]:
# The tan model has some promising results. This file re-does it from scratch

import numpy as np
import pandas as pd
df = pd.read_csv( "/Users/kamyarghiam/Desktop/kalshi_bot/src/data/local/research/11_28_spy_price_to_kalshi_time.csv")

In [ ]:
from data.coledb.coledb import ColeDBInterface
import datetime

date = "2023-11-28"
end_time = "16:00:00"  # 4 pm
datetime_str = f"{date} {end_time}"
dt_object = datetime.datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
formatted_date = dt_object.strftime('%y%b%d').upper()
from data.coledb.coledb import ColeDBInterface
db = ColeDBInterface()
cole_db_path = db.cole_db_storage_path / f"INXZ/{formatted_date}"
market_suffix = list(cole_db_path.iterdir())[0].name
price_threshold = float(market_suffix[1:])*10

In [ ]:
df['x'] -= price_threshold

In [ ]:
df.x = ((((df.x - df.x.min())/(df.x.max()-df.x.min())) - 0.5) * 6)

In [ ]:
# Curve to be fitted (for ONE dataset)
def func( x_time_tup, x0, c):
    xval = x_time_tup[0]
    time = x_time_tup[1]
    return (np.tanh( c * ( xval - x0 )/ (time ) ) + 1)/2

In [ ]:
from scipy.optimize import curve_fit
x1, x2, y1, y2 = min( df.x ), max( df.x ), min( df.y ), max( df.y )
p0 = [ 0, 1000]
params, cv = curve_fit( func, (df.x, df.time), df.y,p0=p0 )

In [ ]:
xtest =  np.linspace( df.x.min(), df.x.max(), 2000 )
time_test = np.linspace( df.time.min(), df.time.max(), 2000 )
ytest = [func((x,t), *params) for x,t in zip(xtest,[2]*2000)]
import matplotlib.pyplot as plt
plt.plot( xtest, ytest, linewidth=2)

ytest = [func((x,t), *params) for x,t in zip(xtest,[5]*2000)]
import matplotlib.pyplot as plt
plt.plot( xtest, ytest, linewidth=2)

ytest = [func((x,t), *params) for x,t in zip(xtest,[9]*2000)]
import matplotlib.pyplot as plt
plt.plot( xtest, ytest, linewidth=2)

plt.scatter( df.x, df.y,)

plt.show()